In [1]:
import json

In [4]:
tex = open("ch01.tex").read()

In [ ]:
def lines_in_env(env,lines):
    """
    Given some lines of tex code and an environment (such as "enumerate" or "align")
    this returns the index of the first line that starts with "\\begin{environment}"
    and the number of lines until "\\end{environment}".
    
    Note: lines[start:start+length] should include both the begin and end lines.
    """
    is_start = lambda line: line.strip().startswith("\\begin{%s}" % env)
    is_end   = lambda line: line.strip().startswith("\\end{%s}"   % env)
    start    = next(i for i,line in enumerate(lines) if is_start(line))
    length   = next(i+1 for i,line in enumerate(lines[start:]) if is_end(line))
    return start,length

In [2]:
def lines_in_env(environment,lines):
    """
    Given some lines of tex code and an environment (such as "enumerate" or "align")
    this returns the index of the first line that starts with "\\begin{environment}"
    and the number of lines until "\\end{environment}".
    
    Note: lines[start:start+length] should include both the begin and end lines.
    """
    is_start = lambda line: line.strip().startswith("\\begin{%s}" % environment)
    is_end   = lambda line: line.strip().startswith("\\end{%s}"   % environment)
    start    = next(i for i,line in enumerate(lines) if is_start(line))
    length   = next(i+1 for i,line in enumerate(lines[start:]) if is_end(line))
    return start,length

def get_cells(tex):
    """
    Given a string of tex, returns the lines of {sagecell}
    environments grouped together by {sagecode} environments
    respecting "%link" commands.
    """
    lines = tex.splitlines()
    blocks = []
    i = 0
    while True:
        try:
            start,length = lines_in_env("sagecode",lines[i:])
            blocks.append({
                "start": i+start,
                "length": length,
            })
            i += start + length
        except StopIteration:
            break
    cells = [[]]
    for block in blocks:
        start,length = block["start"],block["length"]
        j = 0
        while True:
            try:
                cell_start,cell_length = lines_in_env("sagecell",lines[start+j:start+length])
                out_start,out_length = lines_in_env("sageout",lines[start+j:start+length])
                assert cell_start + cell_length == out_start
                if not lines[start+j+cell_start].strip().endswith("%skip"):
                    cells[-1].append({
                        "source": {"start":cell_start+j+start,"length":cell_length},
                        "output": {"start":out_start+j+start,"length":out_length},
                    })
                j += cell_length + out_length
            except StopIteration:
                break
        if not lines[start+length-1].strip().endswith("%link"):
            cells.append([])
    return [a for a in cells if len(a) > 0]


def generate_tests(tex,filename="__FILE__"):
    """
    Given a string of tex, this returns a sage file
    containing all the code with comments for line numbers.
    This can be used to test the code samples.
    """
    lines = tex.splitlines()
    output = "\n\n"
    cells = get_cells(tex)
    print(json.dumps(cells))
    for block in cells:
        output += "\"\"\"\n"
        for cell in block:
            source_start,source_length = cell["source"]["start"],cell["source"]["length"]
            for i in range(source_start + 1,source_start + source_length - 1):
                if lines[i].startswith("\t") or lines[i].startswith("  ") or lines[i-1].strip().endswith("\\"):
                    prefix = "....: "
                else:
                    prefix = "sage: "
                output += prefix + lines[i].replace("\t","    ") + f" # {filename}:{source_start+1}" + "\n"
            out_start,out_length = cell["output"]["start"],cell["output"]["length"]
            for i in range(out_start + 1,out_start + out_length - 1):
                output += lines[i] + "\n"
        output += "\"\"\"\n\n"
    return output

In [3]:
# find files and generate tests
from os import listdir
tests = ""
for f in [f for f in listdir(".") if f.endswith(".tex")][3:4]:
    with open(f) as file:
        tests += f"\n\n\n{generate_tests(file.read(),file.name)}\n\n\n"
# write tests to tmp file
with open("/tmp/test.sage","wt") as file:
    file.write(tests)
#run tests
import os
#print(os.popen('sage -t /tmp/test.sage').read())

[[{"source": {"start": 251, "length": 4}, "output": {"start": 255, "length": 4}}, {"source": {"start": 259, "length": 3}, "output": {"start": 262, "length": 4}}, {"source": {"start": 266, "length": 3}, "output": {"start": 269, "length": 4}}, {"source": {"start": 273, "length": 3}, "output": {"start": 276, "length": 4}}], [{"source": {"start": 283, "length": 4}, "output": {"start": 287, "length": 5}}, {"source": {"start": 292, "length": 3}, "output": {"start": 295, "length": 5}}, {"source": {"start": 300, "length": 3}, "output": {"start": 303, "length": 5}}, {"source": {"start": 308, "length": 3}, "output": {"start": 311, "length": 5}}], [{"source": {"start": 321, "length": 3}, "output": {"start": 324, "length": 5}}, {"source": {"start": 329, "length": 3}, "output": {"start": 332, "length": 5}}], [{"source": {"start": 720, "length": 3}, "output": {"start": 723, "length": 3}}], [{"source": {"start": 730, "length": 3}, "output": {"start": 733, "length": 3}}, {"source": {"start": 736, "len

In [10]:
print("done")

done


In [12]:
print(tests)






"""
sage: A = matrix(ZZ, 2, [-2,2, -3,4]) # ch02.tex:252
sage: S, P, Q = A.smith_form(); S # ch02.tex:252
[1 0]
[0 2]
sage: P*A*Q # ch02.tex:260
[1 0]
[0 2]
sage: P # ch02.tex:267
[0 1]
[1 0]
sage: Q # ch02.tex:274
[ 1 -4]
[ 1 -3]
"""

"""
sage: A = matrix(ZZ, 3, [1,4,9, 16,25,36, 49,64,81]) # ch02.tex:284
sage: S, P, Q = A.smith_form(); S # ch02.tex:284
[ 1  0  0]
[ 0  3  0]
[ 0  0 72]
sage: P*A*Q # ch02.tex:293
[ 1  0  0]
[ 0  3  0]
[ 0  0 72]
sage: P # ch02.tex:301
[  0   0   1]
[  0   1  -1]
[  1 -20 -17]
sage: Q # ch02.tex:309
[  47   74   93]
[ -79 -125 -156]
[  34   54   67]
"""

"""
sage: m = matrix(ZZ, 3, [2..10]); m # ch02.tex:322
[ 2  3  4]
[ 5  6  7]
[ 8  9 10]
sage: m.smith_form()[0] # ch02.tex:330
[1 0 0]
[0 3 0]
[0 0 0]
"""

"""
sage: ZZ.is_noetherian() # ch02.tex:721
True
"""

"""
sage: I = ideal(12,18); I # ch02.tex:731
Principal ideal (6) of Integer Ring
sage: I.is_principal() # ch02.tex:737
True
"""

"""
sage: ZZ.ideal(12,18) # ch02.tex:746
Principal ideal (6) o